## 4. AUTOMATIC DIFFERENTIATION WITH TORCH.AUTOGRAD
https://pytorch.org/tutorials/beginner/basics/autogradqs_tutorial.html

NN(Neural Network)을 training 할 때, 가장 많이 쓰이는 알고리즘이 `back propagation`이다.
Pytorch는 `torch.autograd`라고 하는 내장된 미분 기능을 제공한다. 이는 어떤 computational graph에 대해서도 자동으로 미분을 계산해준다.
- `back propagation` 알고리즘은 loss function의 gradient에 따라 모델의 파라메터를 조정하는 일을 한다.
- `computational graph`는 계산 과정을 그래프로 나타낸 것이다. (https://compmath.korea.ac.kr/appmath2021/BackPropagation.html)

가장 간단한 1층 레이어 NN를 보자. 여기서 input은 x, parameters는 w와 b로 나타난다. 이는 Pytorch에서 아래와 같이 정의할 수 있다.

In [1]:
import torch # pytorch library 불러오기

x = torch.ones(5) # input에 해당하는 tensor
y = torch.zeros(3) # 실제 label 역할을 하는 tensor
w = torch.randn(5, 3, requires_grad=True) # weights에 해당하는 parameter
b = torch.randn(3, requires_grad=True) # bias에 해당하는 parameter
z = torch.matmul(x, w) + b # x와 w를 matrix multiplication을 한 후, bias를 더한다.
loss = torch.nn.functional.binary_cross_entropy_with_logits(z,y)

- requires_grad = True로 되어있는 tensor는, 이 텐서에서 이루어진 모든 연산을 기록하게 된다고 한다. (https://deepinsight.tistory.com/84)

In [12]:
# x와 w의 shape에 관해서, torch.matmul(x, w)가 계산이 되었음.
x.shape, w.shape

(torch.Size([5]), torch.Size([5, 3]))

- ones(n)은 길이가 n인 row로 여겨지는 듯 하다.

## Tensors, Functions and Computational graph

위에서 작성한 코드는 아래의 computational graph를 정의한 것이다.

![Computational graph](https://pytorch.org/tutorials/_images/comp-graph.png)

이 network에서 w와 b는 최적화 과정을 통해 값이 변화되어야 하는 파라메터이다. 이를 위해, w와 b에 대한 loss function의 gradient를 계산할 수 있어야한다.  
이렇게 loss function을 w와 b 각각에 대해서 미분하고 gradient를 구하기 위해, `requires_grad`라는 tensor의 property를 설정할 수 있다.
- `requires_grad`라는 인자는 tensor를 생성할 때 정의하거나, `tensor.requires_grad_(True)` method를 사용해서 설정할 수 있다.

위에서 적욘된 `binary_cross_entropy_with_logits`는 Function 클래스의 함수이다. 이 함수는 forward computation을 진행하고 backward propagation 동안 미분을 계산한다. 여기서 backward propagation 함수에 대한 reference는 텐서의 grad_fn에 저장되어있다.  
아래 코드를 통해서 grad_fn을 확인해보자.

In [13]:
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

Gradient function for z = <AddBackward0 object at 0x0000027802F676A0>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x0000027809F4FEB0>


## Gradients 계산하기

우리의 목적은 NN의 parameters를 최적화하는 것이다. 따라서, parameters에 대한 loss function의 미분, 즉 weights에 대한 $\frac{\partial Loss}{\partial w}$ 와 bias에 대한 $\frac{\partial Loss}{\partial b}$를 계산할 필요가 있다.  
이러한 미분 계산을 위해, `loss.backward()`라는 함수를 사용하고, 이로부터 `w.grad`와 `b.grad`를 통해서 미분 값을 얻을 수 있다.

In [14]:
loss.backward()
print("Derivative of w: ",w.grad)
print("Derivative of b: ",b.grad)

Derivative of w:  tensor([[0.2100, 0.2174, 0.1833],
        [0.2100, 0.2174, 0.1833],
        [0.2100, 0.2174, 0.1833],
        [0.2100, 0.2174, 0.1833],
        [0.2100, 0.2174, 0.1833]])
Derivative of b:  tensor([0.2100, 0.2174, 0.1833])


- Computational graph 상에서, requires_grad = True로 설정된 leaf node만이 `grad` property 만을 얻을 수 있다. graph의 다른 노드들에 대해서는 gradident가 불가능하다.
- `backward`를 통한 Gradient 계산은 성능 문제로 한번씩만 가능하다. 만약 같은 graph에 대해서 여러번 `backward`를 해야한다면, `retain_graph=True`로 설정한 다음 `backward`를 호출한다.


## Gradient tracking 멈추기

Default로 `requires_grad = True`로 텐서는 이들의 계산한 기록들을 tracking하고 gradient 계산을 할 수 있게 도와준다. 하지만, 이것이 모든 경우에 필요한 것은 아니다. 예를들어, 모델이 학습되고 이를 어떤 input data에 적용하고자 할 때, 즉 `forward` 계산만 하고싶은 상황이라 하자. 이 때, `torch.no_grad()`를 통해서 computation tracking(기록)을 멈출 수 있다.

아래 코드는 requires_grad를 False로 바꾸고, 이것이 잘 작동하는지 확인하는 코드이다.

- 그런데 왜 tracking을 멈춰야 하는 것일까? forward 때도 굳이 일일이 `torch.no_grad()`를 선언하지 않고, Gradient tracking을 하게 놔둬도 괜찮지 않을까?
    - gradient tracking에는 메모리 소모가 심한듯 하다(출처: 딥러닝 공부방)

In [18]:
z = torch.matmul(x,w) + b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x,w)+b
print(z.requires_grad)

True
False


- 저 `with` 내부에 있는 코드는 w와 b라는 require_grad=True가 있어도 그 연산 결과를 저장하는 것은 requires_grad = False로 해주는 코드인듯 하다.

위와 같은 결과는 tensor에 `detach()` 함수를 사용해서도 얻을 수 있다.

In [21]:
z = torch.matmul(x,w)+b
print("Before detach(): ",z.requires_grad)
z_det = z.detach()
print("After detach(): ",z_det.requires_grad)

Before detach():  True
After detach():  False


Gradient tracking을 멈추는 것을 고려해볼만한 경우:
1. 모델 내의 몇몇 parameters를 frozen하고싶은 경우
2. Forward 연산만을 수행할 때 계산 속도를 올리고 싶은경우이다. 속도가 올라가는 이유는 gradient를 tracking하지 않은 tensor에 대한 계산이 더 효율적이기 때문이라 한다.

## Computational Graph

개념적으로, autograd가 데이터(tensor)와 모든 수행되는 연산(새로운 텐서 결과에 따라)의 기록을 Directed acyclic graph(DAG) 계속 유지한다. DAG에서는 leaves가 input tensor이고 roots는 output tensors이다. 이 graph를 roots부터 leaves까지 tracking함으로써, 자동으로 `chain rule`을 적용하여 graidents를 계산할 수 있게된다.  

Forward 계산을 할 때, autograd는 다음 두 가지를 동시에 한다:
- 사용자가 요청한 operation을 실행하여 결과 tensor를 계산한다.
- 실행된 operation의 gradient 함수를 DAG에 유지한다.

Backward 계산은 `.backward()`가 DAG의 root에서 호출됬을 때 시작된다. autograd 에서는:
- gradients를 각각의 `.grad_fn`에서부터 계산한다.
- 각각 텐서의 `.grad` 속성에 계산된 gradients를 accumulates한다.
- chain rule을 사용하여, 모든 leaf tensors까지 propagate(전달)한다.


`*` DAG는 튜토리얼 처음에 나온 그래프와 같은 경우로 보면 될듯하다.

`Note`  
Pytorch에서 DAG는 dynamic하다. 여기서 알아둘만한 것은 Graph는 scratch부터 다시 생성된다는 것이다.  
각각의 `.backward()` 호출을 한 후에, autograd는 새로운 graph를 채우기 시작한다. 이는 사용자가 모델에 control flow statements를 사용할 수 있도록 해준다. 만약 필요하다면 사용자는 shape, size와 연산을 매 iteration마다 바꿀 수 있다 한다.

## 튜토리얼 외의 내용

`-` retain_graph 란?

해당 내용을 조사해 보았다.  
먼저, **backward는 한번 씩만 가능하다.** 의 의미는 위의 loss.backward()를 연속으로 호출할 수 없다는 의미이다.  
아래 코드를 보자

- 참고한 블로그:
    - https://hongl.tistory.com/158
    - https://m.blog.naver.com/egg5562/221285463744
    - https://deep-learning-study.tistory.com/305

In [5]:
loss.backward()
loss.backward()

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

- graph에 대한 두 번째 backward를 실행하면 중간의 저장된 값들이 free가 된다.
- 만약 backward를 두번 해야한다면, retain_graph=True로 설정한다.

위에서 x, y, w, b, z, loss를 정의하였고, 이들은 computational graph상에서 모두 연결되어 있는 값들이다. 계산의 진행 방향은 x,w -> b -> z -> y -> loss를 방향으로 연결되어있다.

여기서 loss.backward()를 하여 loss에 대한 미분을 계산하면, loss까지 연결된 x,w,b,z,y,loss 값들을 모두 지워진다는 의미가 된다.  

그런데, 위에서 나온 `retain_graph=True`를 설정하면 graph의 중간 노드값들이 지워지지 않고 그래프가 유지된다는 의미이다.

In [8]:
x = torch.ones(5) # input에 해당하는 tensor
y = torch.zeros(3) # 실제 label 역할을 하는 tensor
w = torch.randn(5, 3, requires_grad=True) # weights에 해당하는 parameter
b = torch.randn(3, requires_grad=True) # bias에 해당하는 parameter
z = torch.matmul(x, w) + b # x와 w를 matrix multiplication을 한 후, bias를 더한다.
loss = torch.nn.functional.binary_cross_entropy_with_logits(z,y)

In [9]:
loss.backward(retain_graph=True)
loss.backward()

- 이렇게 하면 에러가 일어나지 않게된다.

`-` 필자는 retain_graph를 접해본 적이 없어서 언제 사용할까 궁금했는데, 참고 블로그에서 해당 경우를 작성해주셨다.  
해당 내용을 정리해보면,

각 층에 대한 여러 개의 목적함수가 존재하는 multi task learning에 자주 사용된다. 예를들어, loss1, loss2가 있을 때, 각 loss에 대한 기울기를  

loss1.backward(retain_graph=True)
loss2.backward()

로 계산할 수 있다.  

## Test 시작

`-` Gradient 값이 축적되는지 확인하는 과정

`-` backward()를 한번만 한 경우

In [77]:
import torch # pytorch library 불러오기

torch.manual_seed(1) # 항상 같은 코드에 대해서는 같은 결과가 나오도록 고정

x = torch.ones(5) # input에 해당하는 tensor
y = torch.zeros(3) # 실제 label 역할을 하는 tensor
w = torch.randn(5, 3, requires_grad=True) # weights에 해당하는 parameter
b = torch.randn(3, requires_grad=True) # bias에 해당하는 parameter
z = torch.matmul(x, w) + b # x와 w를 matrix multiplication을 한 후, bias를 더한다.
loss = torch.nn.functional.binary_cross_entropy_with_logits(z,y)


loss.backward()

print("Derivative of w: ",w.grad)
print("Derivative of b: ",b.grad)

Derivative of w:  tensor([[0.1083, 0.0369, 0.0726],
        [0.1083, 0.0369, 0.0726],
        [0.1083, 0.0369, 0.0726],
        [0.1083, 0.0369, 0.0726],
        [0.1083, 0.0369, 0.0726]])
Derivative of b:  tensor([0.1083, 0.0369, 0.0726])


`-`  backward()를 두번한 경우

In [79]:
import torch # pytorch library 불러오기

torch.manual_seed(1) # 항상 같은 코드에 대해서는 같은 결과가 나오도록 고정

x = torch.ones(5) # input에 해당하는 tensor
y = torch.zeros(3) # 실제 label 역할을 하는 tensor
w = torch.randn(5, 3, requires_grad=True) # weights에 해당하는 parameter
b = torch.randn(3, requires_grad=True) # bias에 해당하는 parameter
z = torch.matmul(x, w) + b # x와 w를 matrix multiplication을 한 후, bias를 더한다.
loss = torch.nn.functional.binary_cross_entropy_with_logits(z,y)


loss.backward(retain_graph=True)
loss.backward()

print("Derivative of w: ",w.grad)
print("Derivative of b: ",b.grad)

Derivative of w:  tensor([[0.2166, 0.0739, 0.1452],
        [0.2166, 0.0739, 0.1452],
        [0.2166, 0.0739, 0.1452],
        [0.2166, 0.0739, 0.1452],
        [0.2166, 0.0739, 0.1452]])
Derivative of b:  tensor([0.2166, 0.0739, 0.1452])


`-` 만약 축적이 된다면, 이는 `loss.backward()`를 한 결과에 *2를 한 값과 동일할 것이다.

In [81]:
import torch # pytorch library 불러오기

torch.manual_seed(1) # 항상 같은 코드에 대해서는 같은 결과가 나오도록 고정

x = torch.ones(5) # input에 해당하는 tensor
y = torch.zeros(3) # 실제 label 역할을 하는 tensor
w = torch.randn(5, 3, requires_grad=True) # weights에 해당하는 parameter
b = torch.randn(3, requires_grad=True) # bias에 해당하는 parameter
z = torch.matmul(x, w) + b # x와 w를 matrix multiplication을 한 후, bias를 더한다.
loss = torch.nn.functional.binary_cross_entropy_with_logits(z,y)


loss.backward()

print("Derivative of w: ",w.grad*2)
print("Derivative of b: ",b.grad*2)

Derivative of w:  tensor([[0.2166, 0.0739, 0.1452],
        [0.2166, 0.0739, 0.1452],
        [0.2166, 0.0739, 0.1452],
        [0.2166, 0.0739, 0.1452],
        [0.2166, 0.0739, 0.1452]])
Derivative of b:  tensor([0.2166, 0.0739, 0.1452])


- 기존 loss.backward()에 *2를 한 것과 retain_graph=True로 backward를 두번한 결과는 동일하다.

`-` 이번에는 Tutorial 5의 optimizer에서 zero_grad를 했을 때, 축적되는 것이 없어지는지 확인한다.

In [139]:
import torch
from torch import nn # Model build에 필요한 building block을 제공해주는 라이브러리
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data", # root directory
    train = True, # 
    download = True, # 인터넷에서 다운로드 받아 root directory에 저장
    transform=ToTensor() # 데아터를 tensor로 변환
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download = True,
    transform = ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size = 64)
test_dataloader = DataLoader(test_data, batch_size = 64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )
        
    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

`-` optimizer.zero_grad() 를 한 경우

In [300]:
torch.manual_seed(1)
model = NeuralNetwork()


loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)

def train_loop(dataloader, model, loss_fn, optimizer):
    # Compute prediction and loss
    for i in range(10):
        X, y = training_data[1012]
        y = torch.tensor(y).reshape([1])
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        print(f"{i}번째 loss.backward()",list(model.parameters())[3].grad[0:6])
        optimizer.step()
    

train_loop(train_dataloader, model, loss_fn, optimizer)

0번째 loss.backward() tensor([0.0000, 0.0000, 0.0023, 0.0173, 0.0000, 0.0447])
1번째 loss.backward() tensor([0.0000, 0.0000, 0.0023, 0.0173, 0.0000, 0.0447])
2번째 loss.backward() tensor([0.0000, 0.0000, 0.0023, 0.0173, 0.0000, 0.0447])
3번째 loss.backward() tensor([0.0000, 0.0000, 0.0023, 0.0173, 0.0000, 0.0447])
4번째 loss.backward() tensor([0.0000, 0.0000, 0.0022, 0.0172, 0.0000, 0.0447])
5번째 loss.backward() tensor([0.0000, 0.0000, 0.0022, 0.0172, 0.0000, 0.0447])
6번째 loss.backward() tensor([0.0000, 0.0000, 0.0022, 0.0172, 0.0000, 0.0447])
7번째 loss.backward() tensor([0.0000, 0.0000, 0.0022, 0.0172, 0.0000, 0.0447])
8번째 loss.backward() tensor([0.0000, 0.0000, 0.0022, 0.0172, 0.0000, 0.0447])
9번째 loss.backward() tensor([0.0000, 0.0000, 0.0022, 0.0172, 0.0000, 0.0447])


`-` optimizer.zero_grad() 를 하지 않은 경우

retrain_graph=True를 하지 않아도 되는 이유는 loop마다 X,y를 다시 정의해서 computational graph가 새로 생성된 형태여서 그런듯.  
하지만, 모델의 파라메터 자체는 사라지고 생기는 것 없이 계속 유지되므로, w나 b 혹은 model parameters에 저장되어있는 gradient 값은 계속해서 축적이 되는 것 같다.

In [301]:
torch.manual_seed(1)
model = NeuralNetwork()


loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)

def train_loop(dataloader, model, loss_fn, optimizer):
    # Compute prediction and loss
    for i in range(10):
        X, y = training_data[1012]
        y = torch.tensor(y).reshape([1])
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        print(f"{i}번째 loss.backward()",list(model.parameters())[3].grad[0:6])
        optimizer.step()
    

train_loop(train_dataloader, model, loss_fn, optimizer)

0번째 loss.backward() tensor([0.0000, 0.0000, 0.0023, 0.0173, 0.0000, 0.0447])
1번째 loss.backward() tensor([0.0000, 0.0000, 0.0045, 0.0345, 0.0000, 0.0894])
2번째 loss.backward() tensor([0.0000, 0.0000, 0.0068, 0.0518, 0.0000, 0.1341])
3번째 loss.backward() tensor([0.0000, 0.0000, 0.0090, 0.0690, 0.0000, 0.1788])
4번째 loss.backward() tensor([0.0000, 0.0000, 0.0112, 0.0862, 0.0000, 0.2235])
5번째 loss.backward() tensor([0.0000, 0.0000, 0.0135, 0.1035, 0.0000, 0.2681])
6번째 loss.backward() tensor([0.0000, 0.0000, 0.0157, 0.1207, 0.0000, 0.3127])
7번째 loss.backward() tensor([0.0000, 0.0000, 0.0179, 0.1379, 0.0000, 0.3572])
8번째 loss.backward() tensor([0.0000, 0.0000, 0.0202, 0.1552, 0.0000, 0.4017])
9번째 loss.backward() tensor([0.0000, 0.0000, 0.0224, 0.1724, 0.0000, 0.4461])


`-` optimizer.step() 를 하지 않은 경우

In [305]:
torch.manual_seed(1)
model = NeuralNetwork()


loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)

def train_loop(dataloader, model, loss_fn, optimizer):
    # Compute prediction and loss
    for i in range(10):
        X, y = training_data[1012]
        y = torch.tensor(y).reshape([1])
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        print(f"{i}번째 loss.backward()",list(model.parameters())[3].grad[0:6])
        optimizer.step()
    

train_loop(train_dataloader, model, loss_fn, optimizer)

0번째 loss.backward() tensor([0.0000, 0.0000, 0.0023, 0.0173, 0.0000, 0.0447])
1번째 loss.backward() tensor([0.0000, 0.0000, 0.0023, 0.0173, 0.0000, 0.0447])
2번째 loss.backward() tensor([0.0000, 0.0000, 0.0023, 0.0173, 0.0000, 0.0447])
3번째 loss.backward() tensor([0.0000, 0.0000, 0.0023, 0.0173, 0.0000, 0.0447])
4번째 loss.backward() tensor([0.0000, 0.0000, 0.0022, 0.0172, 0.0000, 0.0447])
5번째 loss.backward() tensor([0.0000, 0.0000, 0.0022, 0.0172, 0.0000, 0.0447])
6번째 loss.backward() tensor([0.0000, 0.0000, 0.0022, 0.0172, 0.0000, 0.0447])
7번째 loss.backward() tensor([0.0000, 0.0000, 0.0022, 0.0172, 0.0000, 0.0447])
8번째 loss.backward() tensor([0.0000, 0.0000, 0.0022, 0.0172, 0.0000, 0.0447])
9번째 loss.backward() tensor([0.0000, 0.0000, 0.0022, 0.0172, 0.0000, 0.0447])


결과를 정리해보면, retain_graph=True는 gradient 계산 후에 사라진 computational graph를 복구시키는 것이지만, 저장되있는 gradient에는 영향을 주지 않는다.  
반면에 zero_grad()는 저장되어있는 gradient를 초기화시키는 역할을 한다.